In [14]:
%matplotlib inline  
import matplotlib.pyplot as plt
from pamtra2.libs.singleScattering import scattering
from pamtra2.libs.singleScattering import scattering_utilities as scautils
from pamtra2.libs.singleScattering import mie, rayleigh, tmatrix, ssrg
from pamtra2.libs import refractiveIndex
import numpy as np
np.set_printoptions(precision=3)
rad2deg = 180.0/np.pi

# Angles
t0 = np.pi*0.3 # theta must be within 0 pi
t1 = np.pi*0.6
#t1 = np.pi-t0
p0 = 1.0*np.pi
p1 = p0#0.1*np.pi
#p1 = np.pi+p0
print(p1, p1 // (2.*np.pi))
p1 = p1 - 2.*np.pi*(p1//(2.*np.pi))
print(t0, t1, p0, p1)

if((t0==0.0)and(t1==0.0)):
    Rmat0= np.array([[np.cos(p0),-np.sin(p0)],[-np.sin(p0),-np.cos(p0)]])
    Rmat = np.array([[np.cos(p1),-np.sin(p1)],[-np.sin(p1),-np.cos(p1)]])
elif((t0==0.0)or(p0==p1)):
    Rmat0= np.array([[np.cos(p1-p0),np.sin(p1-p0)],[np.sin(p1-p0),-np.cos(p1-p0)]])
    Rmat = np.array([[1,0],[0,-1]])
elif((t1==0.0)or(p0==p1)):
    Rmat0= np.array([[1,0],[0,-1]])
    Rmat = np.array([[np.cos(p0-p1),np.sin(p0-p1)],[np.sin(p0-p1),-np.cos(p0-p1)]])
else:
    print('do not know what to do')


3.141592653589793 0.0
0.9424777960769379 1.8849555921538759 3.141592653589793 3.141592653589793


In [15]:
# One centimeter particle, S-band 2.8GHz, pure ice sphere 
f = 2.8e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')

R = rayleigh.RayleighScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
M = mie.MieScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
T = tmatrix.TmatrixScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
S = ssrg.SsrgScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1, volume=np.pi*d**3/6.)

from pytmatrix import tmatrix as tm
from pytmatrix import scatter, radar
scatt = tm.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(R.Cext, R.Csca, R.Cabs, R.Cbck)
print(M.Cext, M.Csca, M.Cabs, M.Cbck)
print(S.Cext, S.Csca, S.Cabs, S.Cbck)
print(T.Cext, T.Csca, T.Cabs, T.Cbck)


print(" ")
print("RAY")
print(" ")
print(R.S)
print(" ")
#print(R.S2/(R.S1*np.cos(p1-p0)))
print(" ")
print("MIE")
print(M.S)
print(" ")
#print(M.S2/(M.S1*np.cos(p1-p0)))
print(" ")
print("SSRG")
print(" ")
print(S.S)
print(" ")
print("TMM")
print(T.S)
print(" ")
#print(T.S[1,1]/(T.S[0,0]*np.cos(p1-p0)))
print(" ")
print(scatt.get_S())
print(" ")
print(T.S)
print(" ")


Cext                    Csca                    Cabs                   Cbck            [m**2]
3.42313882614e-12 2.74872213037e-13 3.1482666131e-12 4.12308319556e-13
3.42552492696e-12 2.74936863749e-13 3.15058806321e-12 4.12217233995e-13
3.42307506305e-12 2.748084499522858e-13 3.1482666131e-12 4.12117043789e-13
3.42552500089e-12 2.749368496695048e-13 3.15058815122e-12 4.12217233682e-13
 
RAY
 
[[ nan+nanj  nan+nanj]
 [ nan+nanj  nan+nanj]]
 
 
MIE
[[ nan+nanj  nan+nanj]
 [ nan+nanj  nan+nanj]]
 
 
SSRG
 
[[ nan+nanj  nan+nanj]
 [ nan+nanj  nan+nanj]]
 
TMM
[[  1.065e-07 +9.403e-12j   0.000e+00 +0.000e+00j]
 [  0.000e+00 +0.000e+00j   1.812e-07 +1.599e-11j]]
 
 
[[  1.065e-07 +9.403e-12j   0.000e+00 +0.000e+00j]
 [  0.000e+00 +0.000e+00j   1.812e-07 +1.599e-11j]]
 
[[  1.065e-07 +9.403e-12j   0.000e+00 +0.000e+00j]
 [  0.000e+00 +0.000e+00j   1.812e-07 +1.599e-11j]]
 


In [16]:
a0 = abs(p1 - p0)
a1 = abs(p0 - p1)
print(a0,a1,min(a0,a1),np.cos(min(a0,a1)),np.cos(p1-p0))
print(R.rot_alpha,R.rot_beta,R.scatt_angle)
print(M.rot_alpha,M.rot_beta,M.scatt_angle)
print(S.rot_alpha,S.rot_beta,S.scatt_angle)

0.0 0.0 0.0 1.0 1.0
nan 0.0 0.942477796077
nan 0.0 0.942477796077
nan 0.0 0.942477796077


In [4]:
RS = scautils.amplitude_matrix(R.S)
RZ = scautils.amplitude2mueller(RS)
SS = scautils.amplitude_matrix(S.S)
SZ = scautils.amplitude2mueller(SS)
TS = scautils.amplitude_matrix(T.S)
TZ = scautils.amplitude2mueller(TS)
MS = scautils.amplitude_matrix(M.S)
MZ = scautils.amplitude2mueller(MS)
print(RZ.matrix)
print("")
print(MZ.matrix)
print("")
print(SZ.matrix)
print("")
print("=================")
print("")
print(TZ.matrix)
print("")
print(scatt.get_Z())

[[ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]]

[[ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]
 [ nan  nan  nan  nan]]

[[  3.281e-14   0.000e+00  -0.000e+00   1.926e-34]
 [  0.000e+00   2.654e-14  -1.929e-14  -0.000e+00]
 [ -0.000e+00   1.929e-14   2.654e-14   0.000e+00]
 [  1.926e-34  -0.000e+00   0.000e+00   3.281e-14]]


[[  3.283e-14   0.000e+00  -0.000e+00  -0.000e+00]
 [  0.000e+00   2.656e-14  -1.930e-14  -0.000e+00]
 [ -0.000e+00   1.930e-14   2.656e-14   0.000e+00]
 [ -0.000e+00  -0.000e+00   0.000e+00   3.283e-14]]

[[  3.283e-14   0.000e+00   0.000e+00   0.000e+00]
 [  0.000e+00   2.656e-14  -1.930e-14   0.000e+00]
 [  0.000e+00   1.930e-14   2.656e-14   0.000e+00]
 [  0.000e+00  -0.000e+00   0.000e+00   3.283e-14]]


In [5]:
# One millimeter particle, W-band 94GHz, pure ice sphere 
f = 94e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')
rayC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='TMM', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)
from pytmatrix import tmatrix, scatter, radar
scatt = tmatrix.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)
print([scatter.ext_xsect(scatt),scatter.sca_xsect(scatt),scatter.ext_xsect(scatt)-scatter.sca_xsect(scatt),radar.radar_xsect(scatt)])

Cext                    Csca                    Cabs                   Cbck            [m**2]
(3.5159381620021679e-07, 3.4915146499913131e-07, 2.4423512010854983e-09, 5.2372719749869694e-07, array([[ nan+nanj,  nan+nanj],
       [ nan+nanj,  nan+nanj]]))
(3.8169026989341472e-07, 3.7719995847397647e-07, 4.4903114194382853e-09, 3.0198335883905455e-07, array([[ nan+nanj,  nan+nanj],
       [ nan+nanj,  nan+nanj]]))
(3.8169025547199788e-07, 3.7719994813947296e-07, 4.4903073325249232e-09, 3.0198337240758909e-07, array([[ -2.466e-04 -5.691e-05j,  -8.011e-05 -1.849e-05j],
       [  8.011e-05 +1.849e-05j,  -2.466e-04 -5.691e-05j]]))
[3.8169025547199788e-07, 3.7719994813947296e-07, 4.4903073325249232e-09, 8.0462124145081377e-07]


In [6]:
# One millimeter particle, S-band 2.8GHz, pure ice sphere 
f=2.8e9
n = refractiveIndex.n(frequencies=f,temperatures=263.15,substance='ice')
rayC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='TMM')

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)

Cext                    Csca                    Cabs                   Cbck            [m**2]
(3.1485414853155008e-15, 2.7487221303700193e-19, 3.1482666131024637e-15, 4.1230831955550289e-19, array([[ nan+nanj,  nan+nanj],
       [ nan+nanj,  nan+nanj]]))
(3.1485646987497757e-15, 2.7487285968888451e-19, 3.1482898258900869e-15, 4.1230740899916799e-19, array([[ nan+nanj,  nan+nanj],
       [ nan+nanj,  nan+nanj]]))
(3.1485646291424952e-15, 2.7487285368539355e-19, 3.14828975628881e-15, 4.1230742079816574e-19, array([[  1.811e-10 +1.470e-14j,   0.000e+00 +0.000e+00j],
       [  0.000e+00 +0.000e+00j,   1.811e-10 +1.470e-14j]]))


In [7]:
scatt.get_S()
scatt.get_Z()

array([[  7.079e-08,   0.000e+00,   0.000e+00,   0.000e+00],
       [  0.000e+00,   5.727e-08,  -4.161e-08,   0.000e+00],
       [  0.000e+00,   4.161e-08,   5.727e-08,   0.000e+00],
       [  0.000e+00,  -0.000e+00,   0.000e+00,   7.079e-08]])